In [1]:
import numpy as np
import qcelemental as qcel
import molsym
np.set_printoptions(suppress=True, precision=3, linewidth=600)

In [2]:
# Create Molecule from xyz file

#fn = "test/xyz/ammonia.xyz"
fn = "test/xyz/methane.xyz"
#fn = "test/xyz/benzene.xyz"

mol = molsym.Molecule.from_file(fn)
pg, (paxis, saxis) = molsym.find_point_group(mol)
print(pg)

Td


In [3]:
# Create Molecule from Schema

with open(fn, "r") as lfn:
    file_str = lfn.read()
schema = qcel.models.Molecule.from_data(file_str).dict()
mol = molsym.Molecule.from_schema(schema)
pg, (paxis, saxis) = molsym.find_point_group(mol)
print(pg)

Td


In [4]:
# The coordinates in test/xyz/ammonia.xyz are not symmetric to a tight tolerance

print(f"The default tolerance for detecting symmetry is: {mol.tol}")

# Increase tolerance
mol.tol = 1e-12
pg, (paxis, saxis) = molsym.find_point_group(mol)
print(f"The point group for a tolerance of {mol.tol} is {pg}")
mol.tol = 1e-5

# molsym.symmetrize uses a loose tolerance to detect a potential point group, 
#     then forces the Molecule to that symmetry to near machine precision
mol = molsym.symmetrize(mol)
pg, (paxis, saxis) = molsym.find_point_group(mol)
print(f"The point group for a tolerance of {mol.tol} is {pg} after symmetrization")

The default tolerance for detecting symmetry is: 1e-05
The point group for a tolerance of 1e-12 is Td
The point group for a tolerance of 1e-12 is Td after symmetrization


In [5]:
# Given a valid Schönflies symbol as a string, the symmetry elements (symels) can be generated
molsym.symtext.main.pg_to_symels(pg)

[
 Symbol:          E: [[ 1.00000  0.00000  0.00000],[ 0.00000  1.00000  0.00000],[ 0.00000  0.00000  1.00000]],
 
 Symbol: C_3(alpha): [[ 0.00000  0.00000  1.00000],[ 1.00000  0.00000  0.00000],[ 0.00000  1.00000  0.00000]],
 
 Symbol: C_3^2(alpha): [[ 0.00000  1.00000  0.00000],[ 0.00000  0.00000  1.00000],[ 1.00000  0.00000  0.00000]],
 
 Symbol:  C_3(beta): [[ 0.00000  0.00000  1.00000],[-1.00000  0.00000  0.00000],[ 0.00000 -1.00000  0.00000]],
 
 Symbol: C_3^2(beta): [[ 0.00000 -1.00000  0.00000],[-0.00000  0.00000 -1.00000],[ 1.00000 -0.00000  0.00000]],
 
 Symbol: C_3(gamma): [[ 0.00000  0.00000 -1.00000],[ 1.00000  0.00000  0.00000],[ 0.00000 -1.00000  0.00000]],
 
 Symbol: C_3^2(gamma): [[ 0.00000  1.00000 -0.00000],[ 0.00000  0.00000 -1.00000],[-1.00000 -0.00000  0.00000]],
 
 Symbol: C_3(delta): [[ 0.00000  0.00000 -1.00000],[-1.00000  0.00000  0.00000],[ 0.00000  1.00000  0.00000]],
 
 Symbol: C_3^2(delta): [[ 0.00000 -1.00000 -0.00000],[-0.00000  0.00000  1.00000],[-1.000

In [6]:
# Similarly, the real valued character table of any point group can be generated
molsym.symtext.main.pg_to_chartab(pg)

Character Table for Td
Irreps: ['A1', 'A2', 'E', 'T1', 'T2']
Classes: ['E', '8C_3', '3C_2', '6S_4', '6sigma_d']
Characters:
[[ 1.  1.  1.  1.  1.]
 [ 1.  1.  1. -1. -1.]
 [ 2. -1.  2.  0.  0.]
 [ 3.  1. -1.  1. -1.]
 [ 3. -1. -1. -1.  1.]]

In [7]:
"""
Most of the necessary symmetry information is stored in the Symtext object, such as:
    - A molecule rotated to the symmetry element reference frame
    - Matrices for rotating Cartesian vectors to and from the symmetry element ref. frame
    - List of symmetry elements
    - Character table object
    - Map of all atoms in the Molecule under each symmetry operation
    - Multiplication table of the symmetry operations
"""

symtext = molsym.Symtext.from_molecule(mol)
print(symtext)


MolSym Molecule:
   C       0.00000000    -0.00000000    -0.00000000
   H       1.18465230    -1.18465230     1.18465230
   H      -1.18465230     1.18465230     1.18465230
   H       1.18465230     1.18465230    -1.18465230
   H      -1.18465230    -1.18465230    -1.18465230

Character Table for Td
Irreps: ['A1', 'A2', 'E', 'T1', 'T2']
Classes: ['E', '8C_3', '3C_2', '6S_4', '6sigma_d']
Characters:
[[ 1.  1.  1.  1.  1.]
 [ 1.  1.  1. -1. -1.]
 [ 2. -1.  2.  0.  0.]
 [ 3.  1. -1.  1. -1.]
 [ 3. -1. -1. -1.  1.]]

[
Symbol:          E: [[ 1.00000  0.00000  0.00000],[ 0.00000  1.00000  0.00000],[ 0.00000  0.00000  1.00000]], 
Symbol: C_3(alpha): [[ 0.00000  0.00000  1.00000],[ 1.00000  0.00000  0.00000],[ 0.00000  1.00000  0.00000]], 
Symbol: C_3^2(alpha): [[ 0.00000  1.00000  0.00000],[ 0.00000  0.00000  1.00000],[ 1.00000  0.00000  0.00000]], 
Symbol:  C_3(beta): [[ 0.00000  0.00000  1.00000],[-1.00000  0.00000  0.00000],[ 0.00000 -1.00000  0.00000]], 
Symbol: C_3^2(beta): [[ 0.00000 

In [8]:
"""
The main goal of MolSym is to return Symmetry Adapted Linear Combinations of arbitrary functions.
Built in are SALCs of Cartesian coordinates, internal coordinates, and atomic basis functions.
"""

cart_coords = molsym.salcs.CartesianCoordinates(symtext)
salcs = molsym.salcs.ProjectionOp(symtext, cart_coords)
print(salcs)

SALC from P^A1_00 (3) gamma= 3.464
[-0.    -0.    -0.     0.289 -0.289  0.289 -0.289  0.289  0.289  0.289  0.289 -0.289 -0.289 -0.289 -0.289]
SALC from P^E_00 (3) gamma= 4.899
[ 0.    -0.     0.     0.204 -0.204 -0.408 -0.204  0.204 -0.408  0.204  0.204  0.408 -0.204 -0.204  0.408]
SALC from P^E_10 (3) gamma= 4.899
[-0.    -0.    -0.    -0.354 -0.354 -0.     0.354  0.354 -0.    -0.354  0.354  0.     0.354 -0.354  0.   ]
SALC from P^T2_00 (0) gamma= 1.994
[ 0.501 -0.     0.    -0.433 -0.    -0.    -0.433  0.     0.    -0.433 -0.     0.    -0.433  0.    -0.   ]
SALC from P^T2_10 (0) gamma= 1.994
[ 0.     0.501  0.    -0.    -0.433 -0.     0.    -0.433  0.    -0.    -0.433 -0.    -0.    -0.433  0.   ]
SALC from P^T2_20 (0) gamma= 1.994
[-0.     0.     0.501  0.    -0.    -0.433  0.    -0.    -0.433 -0.     0.    -0.433 -0.     0.    -0.433]
SALC from P^T2_01 (3) gamma= 2.828
[ 0.    -0.     0.    -0.     0.354 -0.354  0.     0.354  0.354 -0.    -0.354  0.354 -0.    -0.354 -0.354]
SALC fro

In [9]:
B = salcs.basis_transformation_matrix
print(B)

[[-0.     0.    -0.     0.501  0.     0.    -0.    -0.    -0.   ]
 [-0.    -0.    -0.    -0.    -0.     0.501  0.     0.    -0.   ]
 [-0.     0.    -0.     0.     0.     0.    -0.     0.501 -0.   ]
 [ 0.289  0.204 -0.354 -0.433 -0.    -0.     0.354  0.    -0.354]
 [-0.289 -0.204 -0.354 -0.     0.354 -0.433 -0.    -0.     0.354]
 [ 0.289 -0.408 -0.    -0.    -0.354 -0.     0.354 -0.433 -0.   ]
 [-0.289 -0.204  0.354 -0.433  0.     0.     0.354  0.     0.354]
 [ 0.289  0.204  0.354  0.     0.354 -0.433  0.    -0.    -0.354]
 [ 0.289 -0.408 -0.     0.     0.354  0.    -0.354 -0.433  0.   ]
 [ 0.289  0.204 -0.354 -0.433 -0.    -0.    -0.354 -0.     0.354]
 [ 0.289  0.204  0.354 -0.    -0.354 -0.433 -0.     0.     0.354]
 [-0.289  0.408  0.     0.     0.354 -0.     0.354 -0.433  0.   ]
 [-0.289 -0.204  0.354 -0.433 -0.    -0.    -0.354 -0.    -0.354]
 [-0.289 -0.204 -0.354  0.    -0.354 -0.433  0.     0.    -0.354]
 [-0.289  0.408  0.    -0.    -0.354  0.    -0.354 -0.433 -0.   ]]


In [10]:
print(B.T@B)

[[ 1. -0. -0. -0. -0.  0. -0.  0. -0.]
 [-0.  1.  0.  0.  0. -0.  0.  0.  0.]
 [-0.  0.  1. -0. -0. -0. -0. -0.  0.]
 [-0.  0. -0.  1.  0.  0. -0. -0.  0.]
 [-0.  0. -0.  0.  1. -0.  0. -0.  0.]
 [ 0. -0. -0.  0. -0.  1.  0. -0.  0.]
 [-0.  0. -0. -0.  0.  0.  1.  0. -0.]
 [ 0.  0. -0. -0. -0. -0.  0.  1. -0.]
 [-0.  0.  0.  0.  0.  0. -0. -0.  1.]]
